# Superconvergence of Pointwise Solutions

This notebook demonstrates superconvergence of pointwise solutions.

In [ ]:
from fenics import *
from mshr import *
import numpy as np

%matplotlib inline

First, define helper functions for solving on mesh.

In [ ]:
def u0_boundary(x, on_boundary):
    return on_boundary

def solve_on_mesh(mesh, f, g, u_D, 
                  boundary=u0_boundary, degree=1):
    '''
    mesh:  1-D, 2-D or 3-D mesh
    f:     force function
    g:     Neumann boundary condition
    u_D:   Dirichlet boundary condition
    '''

    V = FunctionSpace(mesh, 'P', degree)
    u = TrialFunction(V)
    v = TestFunction(V)

    # Define variational problem
    a = dot(grad(u), grad(v))*dx

    L = f * v * dx + g * v * ds
    bc = DirichletBC(V, u_D, boundary)

    # Compute solution
    u = Function(V)
    solve(a == L, u, bc)
    return u
    
def compute_error(u_D, u, mesh):
    '''
    computes errors in L_2 and max pointwise norm
    '''
    # Compute error in L2 norm
    error_L2 = errornorm(u_D, u, 'L2')

    # Compute maximum error at vertices
    vertex_values_u_D = u_D.compute_vertex_values(mesh)
    vertex_values_u = u.compute_vertex_values(mesh)

    error_max = np.max(np.abs(vertex_values_u_D - vertex_values_u))
    
    return error_L2, error_max

## 1-D Example

In [ ]:
f = Expression('-56 * pow(x[0], 6)', degree=8)
u_D = Expression('pow(x[0], 8)', degree=6)
g = Constant(0)

errors_L2 = []
errors_max = []

for n in [8, 16, 32]:
    mesh = UnitIntervalMesh(n)
    
    u = solve_on_mesh(mesh, f, g, u_D, degree=1)
    error_L2, error_max = compute_error(u_D, u, mesh)
    
    errors_L2.append(error_L2)
    errors_max.append(error_max)
    
# Print errors
print('error_L2  =', errors_L2)
print('convergence rate in L2 = ', np.log2(np.divide(errors_L2[0:2], errors_L2[1:3])))
print('error_max =', errors_max)
print('convergence rate in max = ', np.log2(np.divide(errors_max[0:2], errors_max[1:3])))

## 2-D Example

In [ ]:
domain = Rectangle(Point(0,0), Point(1,1))

f = Expression('-12 * x[0] * x[1]', degree=2)
u_D = Expression('pow(x[0], 3)*x[1] + x[0]*pow(x[1], 3)', degree=4)

errors_L2 = []
errors_max = []

for n in [8, 16, 32]:
    mesh = UnitSquareMesh(n, n)

    u = solve_on_mesh(mesh, f, g, u_D, degree=1)
    error_L2, error_max = compute_error(u_D, u, mesh)
    
    if n == 8:
        plot(u)
        plot(mesh)

    errors_L2.append(error_L2)
    errors_max.append(error_max)
    
# Print errors
print('error_L2  =', errors_L2)
print('convergence rate in L2 = ', np.log2(np.divide(errors_L2[0:2], errors_L2[1:3])))
print('error_max =', errors_max)
print('convergence rate in max = ', np.log2(np.divide(errors_max[0:2], errors_max[1:3])))